In [24]:
import selenium
from selenium import webdriver
import os
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
from bs4 import BeautifulSoup

#import openpyxl
#from openpyxl import load_workbook
#from openpyxl import Workbook
#from PIL import Image
#from io import BytesIO
#import requests

In [25]:
print('_'*100)
print('다나와 자동차 판매 실적 크롤러')
print('이 크롤러는 현시점 기준 과거 3년 전 판매실적부터 현재까지 데이터를 수집합니다')
print('이번 달 데이터는 아직 게시되지 않았기 때문에 이번달-1 만큼 데이터가 수집됩니다')
print('='*100)

year_cnt = int(input('1.현재 몇 년도인지 숫자만 입력하세요(예 : 2018) : '))
mon_cnt = int(input('2.현재 몇 월인지 숫자만 입력하세요(예 : 3) : '))

now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
f_dir = '/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'

os.makedirs(f_dir+s+'-'+'다나와 자동차 판매실적')
os.chdir(f_dir+s+'-'+'다나와 자동차 판매실적')

path = '/Users/kibeomkim/Desktop/chromedriver_240/chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')
driver = webdriver.Chrome(path, options=options)
driver.get('http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2020-03-00&MonthTo=')
s_time = time.time()
time.sleep(5)


no = 1
for i in (year_cnt-2,year_cnt-1,year_cnt) : 
    element0 = Select(driver.find_element_by_id('selMonth'))
    element0.select_by_value(str(i))
    time.sleep(5)
    
    os.makedirs('%d년' %i)
    os.chdir('%d년' %i)
    
    
    for a in range(1,13) : 
        element1 = Select(driver.find_element_by_id('selDay'))
        try : 
            element1.select_by_value('0'+str(a))
            time.sleep(5)
        except : 
            element1.select_by_value(str(a))
            time.sleep(5)
        
        
        os.makedirs('%d월 판매실적' %a)
        os.chdir('%d월 판매실적' %a)
        
            
        #페이지 진입 완료
        
        for c in range(1,2) : 
            driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
            time.sleep(2)
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        all_tr_list = soup.find('table', 'recordTable model').find('tbody').find_all('tr')
        
        real_tr_list = []
        for d in all_tr_list : 
            try :
                if d['name'] == 'sub_model' : 
                    pass
            except : 
                real_tr_list.append(d)
        #페이지 정보 수집 여기부터 시작
        
        rank_list1 = []
        img_list1 = []
        name_list1 = []
        num_list1 = []
        rate_list1 = []
        year_list1 = []
        up_down_list1 = []
        year_list2 = []
        up_down_list2 = []
        
        
        
        for e in real_tr_list : 
            img_src = e.find('img')['src']
            img_list1.append(img_src)
            
            ranking = e.find('td', 'rank').get_text().replace('\n', "").strip()
            rank_list1.append(ranking)
            
            name = e.find('a').get_text().replace('\n', "").strip()
            name_list1.append(name)
            
            num = e.find('td','num').get_text().replace('그래프로 보기', "").replace('\n', "").replace(',', "").strip()
            num_list1.append(num)
            
            rate = e.find('td', 'rate right').get_text().replace('\n', "").strip()
            rate_list1.append(rate)
            
            
            right_list = e.find_all('td', 'right')
            
            year1 = right_list[1].get_text().replace(right_list[1].find('span').get_text().strip(), "").replace(',',"").strip()
            year_list1.append(year1)
            
            up_down1 = right_list[1].find('span').get_text().replace('\n', "").strip()
            up_down_list1.append(up_down1)
            
            year2 = right_list[2].get_text().replace(right_list[2].find('span').get_text().strip(), "").replace(',',"").strip()
            year_list2.append(year2)
            
            up_down2 = right_list[2].find('span').get_text().replace('\n', "").strip()
            up_down_list2.append(up_down2)
            
            no +=1

        
        car = pd.DataFrame()
        car['순위'] = rank_list1
        car['모델 명'] = name_list1
        car['판매량'] = num_list1
        car['점유율'] = rate_list1
        car['전월대비'] = year_list1
        car['전월대비 등락'] = up_down_list1
        car['전년대비'] = year_list2
        car['전년대비 등락'] = up_down_list2
        
        fc_name = '%d년 %d월 다나와 자동차 판매실적.csv' %(i, a)
        fx_name = '%d년 %d월 다나와 자동차 판매실적.xlsx' %(i, a)
        car.to_csv(fc_name, encoding='utf-8-sig', index=False)
        car.to_excel(fx_name, index=False)

        
        if mon_cnt == 1 and a == 12 : 
            if i == (year_cnt-1) : 
                break
        elif i == year_cnt :  
            if a == (mon_cnt-1) : 
                break
 
        
        os.chdir(f_dir+s+'-'+'다나와 자동차 판매실적'+'/'+'%d년' %i)
        
    if mon_cnt == 1 and a == 12 : 
        if i == (year_cnt-1) : 
            break
    elif i == year_cnt :  
        if a == (mon_cnt-1) : 
            break
        
    os.chdir(f_dir+s+'-'+'다나와 자동차 판매실적')
        
driver.quit()   

e_time = time.time()       
total_time = e_time - s_time
print('='*100)
print('총 %s건 만큼 데이터를 수집했습니다' %(no-1))
print('총 소요시간은 %s초 입니다' %round(total_time, 2))
print('='*100)   


____________________________________________________________________________________________________
다나와 자동차 판매 실적 크롤러
이 크롤러는 현시점 기준 과거 3년 전 판매실적부터 현재까지 데이터를 수집합니다
이번 달 데이터는 아직 게시되지 않았기 때문에 이번달-1 만큼 데이터가 수집됩니다
1.현재 몇 년도인지 숫자만 입력하세요(예 : 2018) : 2021
2.현재 몇 월인지 숫자만 입력하세요(예 : 3) : 2
엑셀에 사진 첨부 완료


UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7fd0b6635770>